# Imports

In [1]:
import re
import json
import pandas as pd
from sauma.core import Connection
from sqlalchemy import select, insert
from datetime import datetime
from datetime import timedelta

## Setup the connection

In [2]:
c = Connection()

conn = c.connect()

## Show all the schema

In [3]:
c.show_schemas()

[   'information_schema',
    'flexstone',
    'flexstone_db',
    'morning_star',
    'morningstar',
    'mysql',
    'performance_schema',
    'sys',
    'wrds']


## Show all tables in `wrds` schema

In [4]:
c.show_tables('wrds')

[   'Test',
    'alembic_version',
    'mutual_funds',
    'mutualfunds_contact_info',
    'mutualfunds_crsp_cik_map',
    'mutualfunds_crsp_portno_map',
    'mutualfunds_daily_nav',
    'mutualfunds_daily_nav_ret',
    'mutualfunds_daily_returns',
    'mutualfunds_dividends',
    'mutualfunds_front_load',
    'mutualfunds_front_load_det',
    'mutualfunds_front_load_grp',
    'mutualfunds_fund_fees',
    'mutualfunds_fund_flows',
    'mutualfunds_fund_hdr',
    'mutualfunds_fund_hdr_hist',
    'mutualfunds_fund_names',
    'mutualfunds_fund_style',
    'mutualfunds_fund_summary',
    'mutualfunds_fund_summary2',
    'mutualfunds_holdings',
    'mutualfunds_mfdbname',
    'mutualfunds_monthly_nav',
    'mutualfunds_monthly_returns',
    'mutualfunds_monthly_tna',
    'mutualfunds_monthly_tna_ret_nav',
    'mutualfunds_portnomap',
    'mutualfunds_rear_load',
    'mutualfunds_rear_load_det',
    'mutualfunds_rear_load_grp']


## Create Schema `morning_star` and drop table `Test`

In [5]:
sql = "CREATE DATABASE IF NOT EXISTS morning_star"
c.execute(sql)

/home/sahil/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1007, "Can't create database 'morning_star'; database exists")
  result = self._query(query)


In [6]:
sql = "DROP TABLE IF EXISTS morning_star.Test"
conn.execute(sql)

## Use JSON to create Table

In [7]:
with open('test_table.json', 'r') as fl:
    table_dict = json.load(fl)

c.create_table(json.dumps(table_dict))

## Insert test data in table

In [8]:
now = datetime.now().date()

now_time = datetime.now()

df_dict = {
    'id': [1,2,3,4,5,6],
    'text_col': ["text_1", "text_2", "text_3", None, "text_5", "text_6"],
    'int_col': [243, None, 789, None, 909, 1001],
    'float_col_1': [123.0, None, 234.0, None, None, 101.45],
    'float_col_2': [123.324, 234.789, None,None, None, -101.45],
    'id_2': ['id_1', 'id_2', 'id_3', 'id_4', 'id_5', 'id_6'],
    'date_col': [now + timedelta(days=-5), now + timedelta(days=-4), now + timedelta(days=-3),\
                 now + timedelta(days=-2), now + timedelta(days=-1), now],
    'datetime_col': [now_time + timedelta(days=-5), now_time + timedelta(days=-4), now_time + timedelta(days=-3),\
                 now_time + timedelta(days=-2), now_time + timedelta(days=-1), now_time]
}

df = pd.DataFrame(df_dict)

In [9]:
table = c.get_table('Test', 'morning_star') ## SQLAlchemy table

values = df.iloc[0].to_dict()  ## will insert into db

values['id'] = int(values['id'])
values['int_col'] = int(values['int_col'])
values['float_col_1'] = float(values['float_col_1'])
values['float_col_2'] = float(values['float_col_2'])
values['datetime_col'] = values['datetime_col'].to_pydatetime()

stmt = insert(table, values=values)

conn.execute('use morning_star')

In [10]:
conn.execute(stmt) ## execute insert statement to insert into table

## Update the table with pandas DataFrame

In [11]:
c.update_table(table_name = 'Test', dataframe=df.iloc[1:], schema='morning_star', index=False, if_exists='append')

## Test

In [12]:
stmt = select([table]).where(table.columns.id == 1) ## using select

In [13]:
results = conn.execute(stmt)

In [14]:
for result in results:
    print(result)

(1, 'text_1', 243, 123.0, 123.324, 'id_1', datetime.date(2020, 6, 6), datetime.datetime(2020, 6, 6, 15, 40, 12))


In [15]:
stmt = table.select(table.columns.id == 1)   ## using table.select

In [16]:
results = conn.execute(stmt)

In [17]:
for result in results:
    print(result)

(1, 'text_1', 243, 123.0, 123.324, 'id_1', datetime.date(2020, 6, 6), datetime.datetime(2020, 6, 6, 15, 40, 12))


In [18]:
df = c.get_dataframe('Test', 'morning_star')   ## get sql table as dataframe

In [19]:
df

,id,text_col,int_col,float_col_1,float_col_2,id_2,date_col,datetime_col
0,1,text_1,243.0,123.00,123.324,id_1,2020-06-06,2020-06-06 15:40:12
1,2,text_2,NaN,NaN,234.789,id_2,2020-06-07,2020-06-07 15:40:12
2,3,text_3,789.0,234.00,NaN,id_3,2020-06-08,2020-06-08 15:40:12
3,4,None,NaN,NaN,NaN,id_4,2020-06-09,2020-06-09 15:40:12
4,5,text_5,909.0,NaN,NaN,id_5,2020-06-10,2020-06-10 15:40:12
5,6,text_6,1001.0,101.45,-101.450,id_6,2020-06-11,2020-06-11 15:40:12


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 8 columns):
id              6 non-null int64
text_col        5 non-null object
int_col         4 non-null float64
float_col_1     3 non-null float64
float_col_2     3 non-null float64
id_2            6 non-null object
date_col        6 non-null datetime64[ns]
datetime_col    6 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1), object(2)
memory usage: 512.0+ bytes


In [21]:
sql = "SELECT id, text_col, int_col FROM morning_star.Test where id <= 3"
df = c.get_dataframe_from_sql_query(sql)                                ## get sql table as dataframe using query

In [22]:
df

,id,text_col,int_col
0,1,text_1,243.0
1,2,text_2,NaN
2,3,text_3,789.0


## Insert data from `Flat File`

In [23]:
df = pd.read_csv('demo_file.csv')

In [24]:
df.head()

,ticker,"Frequency: A - annual; Q - quarterly; M - monthly; QM - quarterly and monthly; AQM - annual, quarterly, and monthly",Fund Identifier,Date,Latest Month-end NAV,Latest Month-end NAV Date,Latest Month-end TNA,Latest Month-end TNA Date,Income Yield,Calendar Year-to-Date Dividend Sum,...,Date of 52 Week High NAV,52 Week Low NAV,Date of 52 Week Low NAV,Unrealized Appreciation/Depreciation,Unrealized Appreciation/Depreciation Date,Date for Which Asset Composition Applies,Amount of fund invested in Common Stocks,Amount of fund invested in Preferred Stocks,Amount of fund invested in Convertible Bonds,Amount of fund invested in Corporate Bonds
0,APITX,Q,105,20190329,11.75,20190329,24.3,20190329,NaN,NaN,...,20180829,9.83,20181224,NaN,NaN,20190131,75.87,0.0,0,0.0
1,APITX,AQ,105,20171229,14.96,20171229,29.6,20171229,NaN,NaN,...,20171130,12.86,20170103,NaN,NaN,20171031,68.88,0.0,0,0.0
2,APITX,Q,105,20190930,12.47,20190930,24.3,20190930,NaN,NaN,...,20181002,9.83,20181224,NaN,NaN,20190731,77.84,0.0,0,0.0
3,APITX,Q,105,20180329,14.88,20180329,28.9,20180329,NaN,NaN,...,20171130,13.59,20170413,NaN,NaN,20180131,71.64,0.0,0,0.0
4,APITX,Q,105,20180629,14.84,20180629,27.0,20180629,NaN,NaN,...,20171130,14.33,20170706,NaN,NaN,20180430,72.59,0.0,0,0.0


### Preprocess the dataframe

In [25]:
def preprocess(df):
    
    ### rename columns
    columns = [re.sub(r'[-\s/]', '_', col).lower() for col in df.columns]
    columns[1] = 'frequency'
    
    df.columns = columns
    ### extract date columns
    date_cols = [col for col in df.columns if 'date' in col and 'to_date' not in col]
    for col in date_cols:
        df.loc[:, col] = pd.to_datetime(df[col], format='%Y%m%d')

In [26]:
preprocess(df)

In [27]:
c.update_table(table_name='mutual_funds', dataframe=df, schema='morning_star', index=False, if_exists='replace')

## Test

In [28]:
df = c.get_dataframe(table_name='mutual_funds', schema='morning_star')

In [29]:
df.head()

,ticker,frequency,fund_identifier,date,latest_month_end_nav,latest_month_end_nav_date,latest_month_end_tna,latest_month_end_tna_date,income_yield,calendar_year_to_date_dividend_sum,...,date_of_52_week_high_nav,52_week_low_nav,date_of_52_week_low_nav,unrealized_appreciation_depreciation,unrealized_appreciation_depreciation_date,date_for_which_asset_composition_applies,amount_of_fund_invested_in_common_stocks,amount_of_fund_invested_in_preferred_stocks,amount_of_fund_invested_in_convertible_bonds,amount_of_fund_invested_in_corporate_bonds
0,APITX,Q,105,2019-03-29,11.75,2019-03-29,24.3,2019-03-29,NaN,NaN,...,2018-08-29,9.83,2018-12-24,NaN,NaT,2019-01-31,75.87,0.0,0,0.0
1,APITX,AQ,105,2017-12-29,14.96,2017-12-29,29.6,2017-12-29,NaN,NaN,...,2017-11-30,12.86,2017-01-03,NaN,NaT,2017-10-31,68.88,0.0,0,0.0
2,APITX,Q,105,2019-09-30,12.47,2019-09-30,24.3,2019-09-30,NaN,NaN,...,2018-10-02,9.83,2018-12-24,NaN,NaT,2019-07-31,77.84,0.0,0,0.0
3,APITX,Q,105,2018-03-29,14.88,2018-03-29,28.9,2018-03-29,NaN,NaN,...,2017-11-30,13.59,2017-04-13,NaN,NaT,2018-01-31,71.64,0.0,0,0.0
4,APITX,Q,105,2018-06-29,14.84,2018-06-29,27.0,2018-06-29,NaN,NaN,...,2017-11-30,14.33,2017-07-06,NaN,NaT,2018-04-30,72.59,0.0,0,0.0


In [30]:
c.close()   ## close connection